In [1]:
from IPython.display import display
from dotenv import load_dotenv
import pandas as pd
import os, requests

load_dotenv()

True

In [ ]:
test_df = pd.read_csv(os.path.join('test_data', 'test_main.csv'))
API_URL = "https://income-api-32377413295.us-central1.run.app/predict"
API_KEY = os.environ.get("INCOME_API_KEY")
payload = {"inputs": test_df.to_dict(orient="records")}
headers = {"x-api-key": API_KEY}
payload

{'inputs': [{'age': 'Old',
   'workclass': 'Private',
   'education-num': 9,
   'marital-status': 'Married-civ-spouse',
   'occupation': 'Craft-repair',
   'relationship': 'Husband',
   'race': 'White',
   'sex': 'Male',
   'hours-per-week': 40,
   'native-country': 'United-States',
   'capital-profit': 1},
  {'age': 'Middle-aged',
   'workclass': 'Private',
   'education-num': 9,
   'marital-status': 'Married-civ-spouse',
   'occupation': 'Craft-repair',
   'relationship': 'Husband',
   'race': 'White',
   'sex': 'Male',
   'hours-per-week': 40,
   'native-country': 'United-States',
   'capital-profit': 1},
  {'age': 'Middle-aged',
   'workclass': 'Federal-gov',
   'education-num': 15,
   'marital-status': 'Married-civ-spouse',
   'occupation': 'Prof-specialty',
   'relationship': 'Husband',
   'race': 'White',
   'sex': 'Male',
   'hours-per-week': 50,
   'native-country': 'India',
   'capital-profit': 1},
  {'age': 'Young',
   'workclass': 'Private',
   'education-num': 8,
   'marit

In [3]:
try:
    response = requests.post(API_URL, json=payload, headers=headers)
    print(response.status_code)
    predictions = response.json()
    print("Prediction Response Received")
except requests.exceptions.RequestException as e:
    print("Request failed:", e)
    predictions = None

200
Prediction Response Received


In [4]:
if predictions:
    pred_df = pd.DataFrame(predictions['predictions'], columns=["prediction"])
    result_df = pd.concat([test_df, pred_df], axis=1)
    try:
        labels = pd.read_csv(os.path.join('test_data', 'test_labels.csv'))
        result_df["actual"] = labels
        result_df["match"] = result_df["prediction"] == result_df["actual"]
        accuracy = result_df["match"].mean() * 100
        display(result_df)
        print(f"Accuracy on Test Set: {accuracy:.2f}%")
    except Exception as e:
        print("Could not load or match test labels:", e)
else:
    print("No predictions to display.")

,age,workclass,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,capital-profit,prediction,actual,match
0,Old,Private,9,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,1,1,0,False
1,Middle-aged,Private,9,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,1,1,0,False
2,Middle-aged,Federal-gov,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,50,India,1,1,1,True
3,Young,Private,8,Married-civ-spouse,Handlers-cleaners,Own-child,White,Male,40,United-States,1,0,0,True
4,Middle-aged,Private,7,Separated,Machine-op-inspct,Unmarried,Black,Male,40,United-States,1,0,0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6028,Middle-aged,Private,9,Widowed,Adm-clerical,Unmarried,Black,Female,38,United-States,1,0,0,True
6029,Middle-aged,Private,6,Widowed,Other-service,Unmarried,White,Female,40,United-States,1,0,0,True
6030,Middle-aged,Local-gov,10,Married-civ-spouse,Transport-moving,Husband,White,Male,40,United-States,1,1,0,False
6031,Middle-aged,State-gov,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,40,United-States,1,1,0,False


Accuracy on Test Set: 77.87%
